In [1]:
import pandas as pd

data = pd.read_csv('../data/df_undersampling_multilabel.csv', index_col=0)  

In [3]:
data = data.reset_index()
data.head()

,case_title,tag_1,tag_2,gr_no,case_type_total,case_type_count,content,content_clean,wordcount,tokenized,...,case_title_lemmatized_clean,case_title_lemmatized_clean_stopwords,test,CIVIL,COMMERCIAL,CRIMINAL,LABOR,POLITICAL,REMEDIAL,TAX
0,"LEONOR VILLAFLOR VDA. DE VILLANUEVA, PLAINTIFF...",NaN,NaN,G. R. No. L-15737,['CIVIL'],1,"G. R. No. L-15737, February 28, 1962 LEONOR ...",Subject of this direct appeal to us on points ...,1744,"['Subject', 'of', 'this', 'direct', 'appeal', ...",...,leonor villaflor vda de villanueva plaintiff a...,vda appellant juico capacity judicial administ...,vda appellant juico capacity judicial administ...,1,0,0,0,0,0,0
1,"SPOUSES ANDRES OLAR AND SERAPIA GERMANO, SPOUS...",Spouses,Sps,G.R. No. L-47935,['CIVIL'],1,"SECOND DIVISION G.R. No. L-47935, May 05, 197...",Petition for certiorari to annul and set aside...,1476,"['Petition', 'for', 'certiorari', 'to', 'annul...",...,spouses andres olar and serapia germano spouse...,spouses spouses hon cfi spouses lumapas policr...,spouses spouses hon cfi spouses lumapas policr...,1,0,0,0,0,0,0
2,"PHILIPPINE RABBIT BUS LINES, INC., PETITIONER,...",Spouses,Sps,G.R. NO. 160355,"['CIVIL', 'CRIMINAL']",2,"SECOND DIVISION G.R. NO. 160355, May 16, 2005...",This is a petition for review of the Decision[...,3275,"['This', 'is', 'a', 'petition', 'for', 'review...",...,philippine rabbit bus lines petitioner vs heir...,rabbit vs heirs people philippines respondents,rabbit vs heirs people philippines respondents...,1,0,1,0,0,0,0
3,"SPOUSES WILLIAM GUIDANGEN AND MARY GUIDANGEN, ...",Spouses,Sps,G.R. No. 174445,['CIVIL'],1,"FIRST DIVISION G.R. No. 174445, February 15, ...",“The plaintiff must rely on the strength of [h...,4664,"['“', 'The', 'plaintiff', 'must', 'rely', 'on'...",...,spouses william guidangen and mary guidangen p...,spouses guidangen guidangen b wooden respondent,spouses guidangen guidangen b wooden responden...,1,0,0,0,0,0,0
4,"NEGROS DISTRICT CONFERENCE, INC.,\nREVEREND LE...",Spouses,Sps,G.R. No. 57041,['CIVIL'],1,"SECOND DIVISION G.R. No. 57041, October 23, 1...",The United Church of Christ in the Philippines...,2842,"['The', 'United', 'Church', 'of', 'Christ', 'i...",...,negros district conference reverend leumin alo...,negros district conference reverend moderator ...,negros district conference reverend moderator ...,1,0,0,0,0,0,0


### Format Dataset

In [4]:
data['is_civil'] = (data['case_type'] == 'CIVIL')
# data["is_civil"] = df["is_civil"].astype(int)

data['is_commercial'] = (data['case_type'] == 'COMMERCIAL')
# data["is_commercial"] = df["is_commercial"].astype(int)

data['is_criminal'] = (data['case_type'] == 'CRIMINAL')
# data["is_criminal"] = df["is_criminal"].astype(int)

data['is_labor'] = (data['case_type'] == 'LABOR')
# data["is_labor"] = df["is_labor"].astype(int)

data['is_political'] = (data['case_type'] == 'POLITICAL')
# data["is_political"] = df["is_political"].astype(int)

# data['is_remedial'] = (data['case_type'] == 'REMEDIAL')
# data["is_remedial"] = df["is_remedial"].astype(int)

data['is_tax'] = (data['case_type'] == 'TAX')
# data["is_tax"] = df["is_tax"].astype(int)

In [5]:
df = data[['content_clean', 'is_civil', 'is_commercial', 'is_criminal', 'is_labor', 'is_political',  'is_tax']]

In [6]:
df['labels'] = list(zip(df.is_civil.tolist(), df.is_commercial.tolist(), df.is_criminal.tolist(), df.is_labor.tolist(),  df.is_political.tolist(),  df.is_tax.tolist()))

<ipython-input-6-a90776066d10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'] = list(zip(df.is_civil.tolist(), df.is_commercial.tolist(), df.is_criminal.tolist(), df.is_labor.tolist(),  df.is_political.tolist(),  df.is_tax.tolist()))


In [7]:
df = df.rename(columns={"content_clean": 'text'})

### Create Train and Test Dataset

In [8]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.2)

In [10]:
train_df.dtypes

text             object
is_civil           bool
is_commercial      bool
is_criminal        bool
is_labor           bool
is_political       bool
is_tax             bool
labels           object
dtype: object

In [11]:
train_df.shape

(2450, 8)

### Model Training

In [8]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('xlnet', 'xlnet-base-cased', use_cuda=False, num_labels=6)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultiLabelSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream tas

In [9]:
model.train_model(train_df)

(307, 0.24963762628139036)

### Model Prediction

In [10]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [11]:
result

{'LRAP': 0.9109026644915713, 'eval_loss': 0.17275570773265578}

In [12]:
model_outputs

array([[0.77108091, 0.03988045, 0.02475978, 0.01599586, 0.02771435,
        0.01380678],
       [0.14293367, 0.49541849, 0.01002354, 0.03297364, 0.03100283,
        0.05439207],
       [0.16664246, 0.01262887, 0.22881126, 0.02479113, 0.02164016,
        0.00397313],
       ...,
       [0.03171551, 0.01703195, 0.01580507, 0.04320655, 0.94004184,
        0.01575922],
       [0.02271523, 0.02064681, 0.01733292, 0.0167556 , 0.02222091,
        0.96837664],
       [0.0186137 , 0.01842433, 0.02128139, 0.01720255, 0.0183956 ,
        0.95721859]])

In [13]:
wrong_predictions

['NA']

### Model Prediction

In [14]:
predictions, raw_outputs = model.predict(['Avelino Ordono was charged in the municipal court of San Gabriel, La Union with having raped his daughter, Leonora, on October 11, 1970. The verified complaint dated November 7, 1973 was signed by the twenty-four year old victim (Criminal Case No. 104).In support of that complaint, Catalina Balanon Ordono, the mother of Leonora, executed a sworn statement wherein she disclosed that on that same date, October 11th, Leonora had apprised her of the outrage but no denunciation was filed because Avelino Ordono threatened to kill Leonora and Catalina (his daughter and wife, respectively) if they reported the crime to the police.Catalina Ordono in her sworn statement further revealed that her husband had also raped their other daughter, Rosa, on March 25 and April 7, 1973. He was charged in court with that offense.Catalina Ordono said that the rape committed by Avelino Ordono against Leonora was mentioned during the investigation and trial of Avelino Ordono for the rape committed against Rosa Ordono. Catalina\'s statement on this point is as follows:  "Q - Why did you not file the complaint against your husband concerning the incident involving Leonora Ordono?"A - We also narrated the incident during the investigation in the Fiscal\'s Office and also when I testified in court in the case of my daughter Rosa Ordono but then my daughter Leonora Ordono was still in Manila, sir."During the preliminary investigation of the rape committed against Leonora, Catalina manifested that she was no longer afraid to denounce Avelino Ordono because he was already in jail for having raped Rosa Ordono.The case against Avelino Ordono, where Leonora Ordono was the complainant, was elevated to the Court of First Instance of La Union, San Fernando Branch (Criminal Case No. 356). On May 29, 1974 the Fiscal presented Catalina Ordono as the second prosecution witness. After she had stated her personal circumstances, the defense counsel objected to her competency. He invoked the marital disqualification rule found in Rule 130 of the Rules of Court which provides:"SEC. 20.  Disqualification by reason of interest or relationship. — The following persons cannot testify as to matters in which they are interested, directly or indirectly, as herein enumerated:x x x x x x x x x(b) A husband cannot be examined for or against his wife without her consent; nor a wife for or against her husband without his consent, except in a civil case by one against the other, or in a criminal case for a crime committed by one against the other;x x x x x x x x xCounsel claimed that Avelino Ordono had not consented expressly or impliedly to his wife\'s testifying against him.The trial court overruled the objection. After the denial of Avelino Ordono\'s motion for the reconsideration of the adverse ruling, he filed the instant action for certiorari and prohibition. He was allowed to sue in forma pauperis.The issue is whether the rape committed by the husband against his daughter is a crime committed by him against his wife within the meaning of the exception found in the marital disqualification rule.Should the phrase "in a criminal case for a crime committed by one against the other" be restricted to crimes committed by one spouse against the other, such as physical injuries, bigamy, adultery or concubinage, or should it be given a latitudinarian interpretation as referring to any offense causing marital discord?There is a dictum that "where the marital and domestic relations are so strained that there is no more harmony to be preserved nor peace and tranquility which may be disturbed, the reason based upon such harmony and tranquility fails. In such a case identity of interests disappears and the consequent danger of perjury based on that identity is nonexistent. Likewise, in such a situation, the security and confidences of private life which the law aims at protecting will be nothing but ideals which, through their absence, merely leave a void in the unhappy home" (People vs. Francisco, 78 Phil. 694, 704).In the Francisco case, the wife, as a rebuttal witness, was allowed to testify against the husband who was charged with having killed his son and who testified that it was the wife who killed their son.We think that the correct rule, which may be adopted in this jurisdiction, is that laid down in Cargill vs. State, 35 ALR 133, 220 Pac. 64 25 Okl. 314, wherein the court said:"The rule that the injury must amount to a physical wrong upon the person is too narrow; and the rule that any offense remotely or indirectly affecting domestic harmony comes within the exception is too broad. The better rule is that, when an offense directly attacks, or directly and vitally impairs, the conjugal relation, it comes within the exception to the statute that one shall not be a witness against the other except in a criminal prosecution for a crime committed (by) one against the other".Using the criterion thus judiciously enunciated in the Cargill case, it can be conclude that in the law of evidence the rape perpetrated by the father against his daughter is a crime committed by him against his wife (the victim\'s mother).[*]That conclusion is in harmony with the practices and traditions of the Filipino family where, normally, the daughter is close to the mother who, having breast-fed and reared her offspring, is always ready to render her counsel and assistance in time of need. Indeed, when the daughter is in distress or suffers moral or physical pain, she usually utters the word Inay (Mother) before she invokes the name of the Lord. (Madre mia, Dios mio)Thus, in this case, when Avelino Ordono, after having raped his daughter Leonora in the early morning of October 11, 1970, tried to repeat the beastly act in the evening of that date, Leonora shouted "Mother" and, on hearing that word, Avelino desisted.That the rape of the daughter by the father, an undeniably abominable and revolting crime with incestuous implications, positively undermines the connubial relationship, is a proposition too obvious to require much elucidation.In Wilkinson vs. People, 282 Pac. 257, it was held that the wife was a competent witness against the husband in a prosecution for rape committed by the husband against his stepdaughter, who is the wife\'s natural daughter because the crime was "an outrage upon nature in its dearest and tenderest relations as well as a crime against humanity itself". The court adopted the interpretation that "a criminal action or proceeding for a crime committed by one against the other" may refer to a crime where the wife is the individual particularly and directly injured or affected by the crime for which the husband is being prosecuted (See Dill vs. People, 19 Colo. 469, 475, 36 Pac. 229, 232).In State vs. Chambers, 87 Iowa 1, 53 N.W. 1090, it was held under the statutory provision that husband or wife shall in no case be a witness for or against the other, except in a criminal proceeding for a crime committed by one against the other, that the wife was competent to testify against the husband in a case where he was prosecuted for incest committed against his stepdaughter.In State vs. Shultz, 177 Iowa 321, 158 N.W. 539, it was held that the wife may testify against the husband in a case where he was prosecuted for incest committed against their eleven-year old daughter because incest is a "crime committed against the wife". (See Owens vs. State, 32 Neb. 167, 49 N.W. 226; Lord vs. State, 23 N.W. 507, 17 Neb. 526; People vs. Segura, 60 Phil. 933).The trial court did not err in holding that Catalina Ordono could testify against her husband, Avelino Ordono, in the case where he is being tried for having raped their daughter, Leonora.WHEREFORE, the petition for certiorari and prohibition is dismissed. No costs.SO ORDERED.Makalintal, C.J., Castro, Fernando, Teehankee, Barredo, Makasiar, Esguerra, Fernandez, and Muñoz Palma, JJ., concur.Antonio, J., is on sick leave.[*] As noted by Wigmore, the marital disqualification rule consists of two parts: (1) the incapacity of one spouse to testify for the other, a disqualification designed to obviate perjury, and (2) the privilege of one spouse not to testify against the other, a right designed to present domestic disunion and unhappiness (U.S. vs. Concepcion, 31 Phil. 182, 187; 2 Wigmore on Evidence 731).Wigmore notes that the privilege has no longer any good reason for retention. "In an age which has so far rationalized, depolarized, and depolarized the marital relation and the spirit of Femininity as to be willing to enact complete legal and political equality and independence of man and woman, this marital privilege is the merest anachronism, in legal theory, and an indefensible obstruction to truth, in practice". After noting that some States had abolished the privilege, the American Bar Association\'s Committee on the Improvement of the Law of Evidence in 1937-38 recommended its abolition (8 Wigmore on Evidence 232).'])

In [15]:
predictions

[[0, 0, 1, 0, 0, 0]]

In [16]:
raw_outputs

array([[0.03339857, 0.010126  , 0.78424799, 0.02217317, 0.01396317,
        0.00757939]])

### Reuse Model Training

In [4]:
from simpletransformers.classification import MultiLabelClassificationModel
model2 = MultiLabelClassificationModel(
    "xlnet", "outputs/checkpoint-307-epoch-1", use_cuda=False, num_labels=6
)

In [19]:
from sklearn.model_selection import train_test_split

train_df_2, eval_df_2 = train_test_split(df, test_size=0.2)

In [20]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df_2)

In [21]:
result

{'LRAP': 0.9222675367047307, 'eval_loss': 0.1584884635620303}

In [22]:
model_outputs

array([[0.05792563, 0.01484402, 0.02056894, 0.03317257, 0.57441074,
        0.02284676],
       [0.02985523, 0.01134961, 0.8401491 , 0.01833499, 0.01458666,
        0.00977732],
       [0.0865494 , 0.04126722, 0.01272155, 0.01167344, 0.03097227,
        0.46078286],
       ...,
       [0.02681619, 0.03354339, 0.01855225, 0.94206673, 0.01685225,
        0.00985863],
       [0.02492914, 0.00768173, 0.54617268, 0.01389993, 0.05503519,
        0.00758238],
       [0.03365991, 0.01456571, 0.01215248, 0.76921457, 0.05686301,
        0.00793877]])

In [23]:
wrong_predictions

['NA']

In [27]:
predictions = model2.predict(['Avelino Ordono was charged in the municipal court of San Gabriel, La Union with having raped his daughter, Leonora, on October 11, 1970. The verified complaint dated November 7, 1973 was signed by the twenty-four year old victim (Criminal Case No. 104).In support of that complaint, Catalina Balanon Ordono, the mother of Leonora, executed a sworn statement wherein she disclosed that on that same date, October 11th, Leonora had apprised her of the outrage but no denunciation was filed because Avelino Ordono threatened to kill Leonora and Catalina (his daughter and wife, respectively) if they reported the crime to the police.Catalina Ordono in her sworn statement further revealed that her husband had also raped their other daughter, Rosa, on March 25 and April 7, 1973. He was charged in court with that offense.Catalina Ordono said that the rape committed by Avelino Ordono against Leonora was mentioned during the investigation and trial of Avelino Ordono for the rape committed against Rosa Ordono. Catalina\'s statement on this point is as follows:  "Q - Why did you not file the complaint against your husband concerning the incident involving Leonora Ordono?"A - We also narrated the incident during the investigation in the Fiscal\'s Office and also when I testified in court in the case of my daughter Rosa Ordono but then my daughter Leonora Ordono was still in Manila, sir."During the preliminary investigation of the rape committed against Leonora, Catalina manifested that she was no longer afraid to denounce Avelino Ordono because he was already in jail for having raped Rosa Ordono.The case against Avelino Ordono, where Leonora Ordono was the complainant, was elevated to the Court of First Instance of La Union, San Fernando Branch (Criminal Case No. 356). On May 29, 1974 the Fiscal presented Catalina Ordono as the second prosecution witness. After she had stated her personal circumstances, the defense counsel objected to her competency. He invoked the marital disqualification rule found in Rule 130 of the Rules of Court which provides:"SEC. 20.  Disqualification by reason of interest or relationship. — The following persons cannot testify as to matters in which they are interested, directly or indirectly, as herein enumerated:x x x x x x x x x(b) A husband cannot be examined for or against his wife without her consent; nor a wife for or against her husband without his consent, except in a civil case by one against the other, or in a criminal case for a crime committed by one against the other;x x x x x x x x xCounsel claimed that Avelino Ordono had not consented expressly or impliedly to his wife\'s testifying against him.The trial court overruled the objection. After the denial of Avelino Ordono\'s motion for the reconsideration of the adverse ruling, he filed the instant action for certiorari and prohibition. He was allowed to sue in forma pauperis.The issue is whether the rape committed by the husband against his daughter is a crime committed by him against his wife within the meaning of the exception found in the marital disqualification rule.Should the phrase "in a criminal case for a crime committed by one against the other" be restricted to crimes committed by one spouse against the other, such as physical injuries, bigamy, adultery or concubinage, or should it be given a latitudinarian interpretation as referring to any offense causing marital discord?There is a dictum that "where the marital and domestic relations are so strained that there is no more harmony to be preserved nor peace and tranquility which may be disturbed, the reason based upon such harmony and tranquility fails. In such a case identity of interests disappears and the consequent danger of perjury based on that identity is nonexistent. Likewise, in such a situation, the security and confidences of private life which the law aims at protecting will be nothing but ideals which, through their absence, merely leave a void in the unhappy home" (People vs. Francisco, 78 Phil. 694, 704).In the Francisco case, the wife, as a rebuttal witness, was allowed to testify against the husband who was charged with having killed his son and who testified that it was the wife who killed their son.We think that the correct rule, which may be adopted in this jurisdiction, is that laid down in Cargill vs. State, 35 ALR 133, 220 Pac. 64 25 Okl. 314, wherein the court said:"The rule that the injury must amount to a physical wrong upon the person is too narrow; and the rule that any offense remotely or indirectly affecting domestic harmony comes within the exception is too broad. The better rule is that, when an offense directly attacks, or directly and vitally impairs, the conjugal relation, it comes within the exception to the statute that one shall not be a witness against the other except in a criminal prosecution for a crime committed (by) one against the other".Using the criterion thus judiciously enunciated in the Cargill case, it can be conclude that in the law of evidence the rape perpetrated by the father against his daughter is a crime committed by him against his wife (the victim\'s mother).[*]That conclusion is in harmony with the practices and traditions of the Filipino family where, normally, the daughter is close to the mother who, having breast-fed and reared her offspring, is always ready to render her counsel and assistance in time of need. Indeed, when the daughter is in distress or suffers moral or physical pain, she usually utters the word Inay (Mother) before she invokes the name of the Lord. (Madre mia, Dios mio)Thus, in this case, when Avelino Ordono, after having raped his daughter Leonora in the early morning of October 11, 1970, tried to repeat the beastly act in the evening of that date, Leonora shouted "Mother" and, on hearing that word, Avelino desisted.That the rape of the daughter by the father, an undeniably abominable and revolting crime with incestuous implications, positively undermines the connubial relationship, is a proposition too obvious to require much elucidation.In Wilkinson vs. People, 282 Pac. 257, it was held that the wife was a competent witness against the husband in a prosecution for rape committed by the husband against his stepdaughter, who is the wife\'s natural daughter because the crime was "an outrage upon nature in its dearest and tenderest relations as well as a crime against humanity itself". The court adopted the interpretation that "a criminal action or proceeding for a crime committed by one against the other" may refer to a crime where the wife is the individual particularly and directly injured or affected by the crime for which the husband is being prosecuted (See Dill vs. People, 19 Colo. 469, 475, 36 Pac. 229, 232).In State vs. Chambers, 87 Iowa 1, 53 N.W. 1090, it was held under the statutory provision that husband or wife shall in no case be a witness for or against the other, except in a criminal proceeding for a crime committed by one against the other, that the wife was competent to testify against the husband in a case where he was prosecuted for incest committed against his stepdaughter.In State vs. Shultz, 177 Iowa 321, 158 N.W. 539, it was held that the wife may testify against the husband in a case where he was prosecuted for incest committed against their eleven-year old daughter because incest is a "crime committed against the wife". (See Owens vs. State, 32 Neb. 167, 49 N.W. 226; Lord vs. State, 23 N.W. 507, 17 Neb. 526; People vs. Segura, 60 Phil. 933).The trial court did not err in holding that Catalina Ordono could testify against her husband, Avelino Ordono, in the case where he is being tried for having raped their daughter, Leonora.WHEREFORE, the petition for certiorari and prohibition is dismissed. No costs.SO ORDERED.Makalintal, C.J., Castro, Fernando, Teehankee, Barredo, Makasiar, Esguerra, Fernandez, and Muñoz Palma, JJ., concur.Antonio, J., is on sick leave.[*] As noted by Wigmore, the marital disqualification rule consists of two parts: (1) the incapacity of one spouse to testify for the other, a disqualification designed to obviate perjury, and (2) the privilege of one spouse not to testify against the other, a right designed to present domestic disunion and unhappiness (U.S. vs. Concepcion, 31 Phil. 182, 187; 2 Wigmore on Evidence 731).Wigmore notes that the privilege has no longer any good reason for retention. "In an age which has so far rationalized, depolarized, and depolarized the marital relation and the spirit of Femininity as to be willing to enact complete legal and political equality and independence of man and woman, this marital privilege is the merest anachronism, in legal theory, and an indefensible obstruction to truth, in practice". After noting that some States had abolished the privilege, the American Bar Association\'s Committee on the Improvement of the Law of Evidence in 1937-38 recommended its abolition (8 Wigmore on Evidence 232).'])

In [28]:
predictions[0][0]

[0, 0, 1, 0, 0, 0]

In [24]:
predictions[1][0]

array([0.03339857, 0.010126  , 0.78424799, 0.02217317, 0.01396317,
       0.00757939])

In [12]:
raw_outputs[0]

array([0.03339857, 0.010126  , 0.78424799, 0.02217317, 0.01396317,
       0.00757939])

In [26]:
from sklearn.model_selection import train_test_split

train_df_3, eval_df_3 = train_test_split(df, test_size=0.2)

In [27]:
result_2, model_outputs_2, wrong_predictions_2 = model.eval_model(eval_df_3)

In [28]:
print(result_2, model_outputs_2, wrong_predictions_2)

{'LRAP': 0.930342577487765, 'eval_loss': 0.15474733371626248} [[0.08063833 0.03127633 0.02716441 0.03851993 0.03134582 0.74480569]
 [0.77555144 0.03937038 0.01453995 0.01911417 0.04273732 0.02329081]
 [0.05134631 0.0060065  0.23588265 0.01783723 0.02926497 0.01461087]
 ...
 [0.03086768 0.02523039 0.01663977 0.93858039 0.01635276 0.00999285]
 [0.02549833 0.02170361 0.01878417 0.01798782 0.0271228  0.97697949]
 [0.42716098 0.02244153 0.01215147 0.02026433 0.17853558 0.01001685]] ['NA']


In [29]:
from sklearn.model_selection import train_test_split

train_df_4, eval_df_4 = train_test_split(df, test_size=0.2)

In [ ]:
result_4, model_outputs_4, wrong_predictions_4 = model.eval_model(eval_df_4)

In [ ]:
print(result_4, model_outputs_4, wrong_predictions_4)